In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
warnings.simplefilter("ignore")
from config.paths_config import *

In [51]:
data=pd.read_csv(r"C:\Users\hrith\OneDrive\Documents\mlops_tutorial\mlops_beginner_to_advanced\hotel_reservations_project\artifacts\raw\train.csv")
data.head(1)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN25630,2,1,2,1,Meal Plan 1,0,Room_Type 1,26,2017,10,17,Online,0,0,0,161.0,0,Not_Canceled


In [52]:
data.drop(columns=['Booking_ID', 'arrival_year'],inplace=True)
data.shape

(29020, 17)

In [53]:
data.drop_duplicates(inplace=True)
data.shape

(21359, 17)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21359 entries, 0 to 29019
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          21359 non-null  int64  
 1   no_of_children                        21359 non-null  int64  
 2   no_of_weekend_nights                  21359 non-null  int64  
 3   no_of_week_nights                     21359 non-null  int64  
 4   type_of_meal_plan                     21359 non-null  object 
 5   required_car_parking_space            21359 non-null  int64  
 6   room_type_reserved                    21359 non-null  object 
 7   lead_time                             21359 non-null  int64  
 8   arrival_month                         21359 non-null  int64  
 9   arrival_date                          21359 non-null  int64  
 10  market_segment_type                   21359 non-null  object 
 11  repeated_guest      

In [55]:
cat_cols=['type_of_meal_plan','room_type_reserved','market_segment_type','booking_status','required_car_parking_space', 'repeated_guest']
num_cols=[col for col in data.columns if col not in cat_cols]

In [56]:
cat_cols

['type_of_meal_plan',
 'room_type_reserved',
 'market_segment_type',
 'booking_status',
 'required_car_parking_space',
 'repeated_guest']

In [57]:
num_cols

['no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'lead_time',
 'arrival_month',
 'arrival_date',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [58]:
data.isna().sum()

no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

In [59]:
# Univariate analysis : check distribution of each feature
# Bivariate analysis : check how each feature is affecting the Y variable, check correlation among features

In [60]:
# Data Cleaning and processing

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21359 entries, 0 to 29019
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          21359 non-null  int64  
 1   no_of_children                        21359 non-null  int64  
 2   no_of_weekend_nights                  21359 non-null  int64  
 3   no_of_week_nights                     21359 non-null  int64  
 4   type_of_meal_plan                     21359 non-null  object 
 5   required_car_parking_space            21359 non-null  int64  
 6   room_type_reserved                    21359 non-null  object 
 7   lead_time                             21359 non-null  int64  
 8   arrival_month                         21359 non-null  int64  
 9   arrival_date                          21359 non-null  int64  
 10  market_segment_type                   21359 non-null  object 
 11  repeated_guest      

In [62]:
#data['type_of_meal_plan']=np.where(data['type_of_meal_plan']=='Not Selected', 0, data['type_of_meal_plan'])
data['type_of_meal_plan'].value_counts()

type_of_meal_plan
Meal Plan 1     16759
Not Selected     3627
Meal Plan 2       969
Meal Plan 3         4
Name: count, dtype: int64

In [63]:
def modify_meal_plan(x):
    if x=='Not Selected':
        return 0

    if x=='Meal Plan 1':
        return 1

    if x=='Meal Plan 2':
        return 2

    if x=='Meal Plan 3':
        return 3

In [64]:
data['type_of_meal_plan']=data['type_of_meal_plan'].apply(lambda x: modify_meal_plan(x))
data['type_of_meal_plan'].value_counts()

type_of_meal_plan
1    16759
0     3627
2      969
3        4
Name: count, dtype: int64

In [65]:
data['room_type_reserved'].value_counts(normalize=True)


room_type_reserved
Room_Type 1    0.719837
Room_Type 4    0.206143
Room_Type 6    0.035489
Room_Type 2    0.023409
Room_Type 5    0.008896
Room_Type 7    0.006040
Room_Type 3    0.000187
Name: proportion, dtype: float64

In [66]:
data['room_type_reserved']=data['room_type_reserved'].apply(lambda x: int(x.split(' ')[-1]) )
data['room_type_reserved'].value_counts(normalize=True)

room_type_reserved
1    0.719837
4    0.206143
6    0.035489
2    0.023409
5    0.008896
7    0.006040
3    0.000187
Name: proportion, dtype: float64

In [67]:
data['booking_status'].value_counts(normalize=True)

booking_status
Not_Canceled    0.713095
Canceled        0.286905
Name: proportion, dtype: float64

In [68]:
data['booking_status']=data['booking_status'].apply(lambda x: 1 if x=='Canceled' else 0)
data['booking_status'].value_counts(normalize=True)

booking_status
0    0.713095
1    0.286905
Name: proportion, dtype: float64

In [69]:
data['market_segment_type'].value_counts(normalize=True)

market_segment_type
Online           0.764268
Offline          0.162508
Corporate        0.055808
Complementary    0.013624
Aviation         0.003792
Name: proportion, dtype: float64

In [70]:
def modify_market_segment_type(x):
    if x=='Online':
        return 0

    if x=='Offline':
        return 1

    if x=='Corporate':
        return 2

    if x=='Complementary':
        return 3
    
    if x=='Aviation':
        return 4

In [71]:
data['market_segment_type']=data['market_segment_type'].apply(lambda x: modify_market_segment_type(x))
data['market_segment_type'].value_counts(normalize=True)

market_segment_type
0    0.764268
1    0.162508
2    0.055808
3    0.013624
4    0.003792
Name: proportion, dtype: float64

In [72]:
data.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,1,2,1,1,0,1,26,10,17,0,0,0,0,161.00,0,0
1,2,1,1,1,1,0,1,98,7,16,0,0,0,0,121.50,2,0
2,2,0,0,3,1,0,1,433,9,8,1,0,0,0,70.00,0,1
3,2,0,2,5,1,0,1,195,8,8,1,0,0,0,72.25,0,0
4,1,0,0,2,1,0,1,188,6,15,1,0,0,0,130.00,0,1


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21359 entries, 0 to 29019
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          21359 non-null  int64  
 1   no_of_children                        21359 non-null  int64  
 2   no_of_weekend_nights                  21359 non-null  int64  
 3   no_of_week_nights                     21359 non-null  int64  
 4   type_of_meal_plan                     21359 non-null  int64  
 5   required_car_parking_space            21359 non-null  int64  
 6   room_type_reserved                    21359 non-null  int64  
 7   lead_time                             21359 non-null  int64  
 8   arrival_month                         21359 non-null  int64  
 9   arrival_date                          21359 non-null  int64  
 10  market_segment_type                   21359 non-null  int64  
 11  repeated_guest      

In [74]:
################################# checking for multicollinearity #####################################

In [75]:
X=add_constant(data)
X.shape

(21359, 18)

In [76]:
vif_data=pd.DataFrame({'feature':X.columns})
vif_data.shape

(18, 1)

In [77]:
vif_data['VIF']=[ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]
vif_data.head()

,feature,VIF
0,const,34.569984
1,no_of_adults,1.341775
2,no_of_children,1.279622
3,no_of_weekend_nights,1.073545
4,no_of_week_nights,1.124571


In [78]:
################# All VIF values for features are less than 2 , hence there is very less correlation among features

In [79]:
########################################## treating columns with high values ##############################################

In [80]:
max_vals=data.max().reset_index()
max_vals

,index,0
0,no_of_adults,4.0
1,no_of_children,10.0
2,no_of_weekend_nights,6.0
3,no_of_week_nights,17.0
4,type_of_meal_plan,3.0
5,required_car_parking_space,1.0
6,room_type_reserved,7.0
7,lead_time,443.0
8,arrival_month,12.0
9,arrival_date,31.0


In [81]:
##################### max values treatement is not required ##########################

In [82]:
############################ checking skewness ###################################

In [83]:
skewness=data.skew().reset_index()
skewness

,index,0
0,no_of_adults,-0.305652
1,no_of_children,4.165696
2,no_of_weekend_nights,0.636637
3,no_of_week_nights,1.553657
4,type_of_meal_plan,-0.516680
5,required_car_parking_space,4.538315
6,room_type_reserved,1.392145
7,lead_time,1.405258
8,arrival_month,-0.293266
9,arrival_date,0.010333


In [84]:
skew_cols=skewness[skewness[0]>5]['index'].to_list()
skew_cols

['repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled']

In [85]:
for col in skew_cols:
    data[col]=np.log1p(data[col])

In [86]:
skewness_new=data.skew()
skewness_new

no_of_adults                            -0.305652
no_of_children                           4.165696
no_of_weekend_nights                     0.636637
no_of_week_nights                        1.553657
type_of_meal_plan                       -0.516680
required_car_parking_space               4.538315
room_type_reserved                       1.392145
lead_time                                1.405258
arrival_month                           -0.293266
arrival_date                             0.010333
market_segment_type                      2.333046
repeated_guest                           5.282330
no_of_previous_cancellations            13.270580
no_of_previous_bookings_not_canceled     7.619850
avg_price_per_room                       0.542888
no_of_special_requests                   0.922373
booking_status                           0.942305
dtype: float64

In [87]:
###############################################################

In [88]:
data.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,1,2,1,1,0,1,26,10,17,0,0.0,0.0,0.0,161.00,0,0
1,2,1,1,1,1,0,1,98,7,16,0,0.0,0.0,0.0,121.50,2,0
2,2,0,0,3,1,0,1,433,9,8,1,0.0,0.0,0.0,70.00,0,1
3,2,0,2,5,1,0,1,195,8,8,1,0.0,0.0,0.0,72.25,0,0
4,1,0,0,2,1,0,1,188,6,15,1,0.0,0.0,0.0,130.00,0,1


In [1]:
import numpy as np
from config.paths_config import MODEL_OUTPUT_PATH
import joblib

In [3]:
df_processed=pd.read_csv(r"C:\Users\hrith\OneDrive\Documents\mlops_tutorial\mlops_beginner_to_advanced\hotel_reservations_project\artifacts\processed\processed_train.csv")
df_processed.shape

(30462, 11)

In [4]:
df_processed.head(1)

,lead_time,no_of_special_requests,avg_price_per_room,arrival_date,arrival_month,market_segment_type,no_of_week_nights,no_of_weekend_nights,type_of_meal_plan,room_type_reserved,booking_status
0,26,0,161.0,17,10,4,1,2,0,0,1


In [5]:
model=joblib.load(r'C:\Users\hrith\OneDrive\Documents\mlops_tutorial\mlops_beginner_to_advanced\hotel_reservations_project\artifacts\models\lgbm_model.pkl')

In [6]:
df_processed['prediction']=model.predict(df_processed[['lead_time', 
                              'no_of_special_requests', 
                              'avg_price_per_room', 
                              'arrival_date', 
                              'arrival_month', 
                              'market_segment_type',
                              'no_of_week_nights',
                              'no_of_weekend_nights', 
                              'type_of_meal_plan',
                              'room_type_reserved']])

In [7]:
df_processed[df_processed['prediction']==1].head(1)

,lead_time,no_of_special_requests,avg_price_per_room,arrival_date,arrival_month,market_segment_type,no_of_week_nights,no_of_weekend_nights,type_of_meal_plan,room_type_reserved,booking_status,prediction
0,26,0,161.0,17,10,4,1,2,0,0,1,1


In [8]:
df_processed[df_processed['prediction']==0].head(1)

,lead_time,no_of_special_requests,avg_price_per_room,arrival_date,arrival_month,market_segment_type,no_of_week_nights,no_of_weekend_nights,type_of_meal_plan,room_type_reserved,booking_status,prediction
2,433,0,70.0,8,9,3,3,0,0,0,0,0


In [9]:
#